<a href="https://colab.research.google.com/github/usesnames/Reviews_analyzer/blob/main/Reviews_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq google-play-scraper

     |████████████████████████████████| 51kB 2.4MB/s 


In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

from google_play_scraper import Sort, reviews, app, reviews_all

In [ ]:
app_name = 'com.latuabancaperandroid'

## Scraping App Information

Let's scrape the info for our app:

In [ ]:
app_infos = []

info = app(app_name, lang='en', country='us')
del info['comments']

We got the info for our app. Let's write a helper function that prints JSON objects a bit better:

In [ ]:
def print_json(json_object):
  json_str = json.dumps(
    json_object, 
    indent=2, 
    sort_keys=True, 
    default=str
  )
  print(json_str)

Here are the app information:

In [ ]:
print_json(info)

{
  "adSupported": null,
  "androidVersion": "5.0",
  "androidVersionText": "5.0 and up",
  "appId": "com.latuabancaperandroid",
  "containsAds": false,
  "contentRating": "Everyone",
  "contentRatingDescription": null,
  "currency": "USD",
  "description": "Semplice, veloce, personale: \u201cIntesa Sanpaolo Mobile\u201d \u00e8 l'applicazione di Intesa Sanpaolo per la banca di tutti i giorni. \r\nCon \u201cIntesa Sanpaolo Mobile\u201d puoi:\r\n- consultare il conto e le carte, fare bonifici, ricariche, bollettini e altri pagamenti;\r\n- gestire le tue carte di credito, debito e prepagate nominative, configurando limiti, opzioni geocontrol, carte virtuali associate e sospendendo o bloccando le carte in caso di emergenza;\r\n- aggregare in un'unica vista tutti tuoi conti e le tue carte, anche presso le altre banche, con XME Banks;\r\n- ricevere assistenza personalizzata dalla filiale online con un semplice \"shake\" in ogni momento della navigazione;\r\n- pagare nei negozi convenzionati 

In [ ]:
app_reviews = []

for score in list(range(1, 6)):
  rvs, _ = reviews(
    app_name,
    lang='it',
    country='it',
    sort=Sort.NEWEST,
    count= 4000, 
    filter_score_with=score
  )
  for r in rvs:
    r['appId'] = app_name
  app_reviews.extend(rvs)

In [ ]:
print_json(app_reviews[0])

{
  "appId": "com.latuabancaperandroid",
  "at": "2021-03-23 16:22:25",
  "content": "A me piace perch\u00e9 mi consente di tenere costantemente aggiornato la mie situazione finanziaria",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": "2.18.2",
  "reviewId": "gp:AOqpTOGtJEUZEiJym1GYT0FxJnoItJzngbCFDcTOPkwka6B6T9KMX3gUfbYa0cIhHD3yvRXcT92rgXEooirBNQ",
  "score": 1,
  "thumbsUpCount": 0,
  "userImage": "https://play-lh.googleusercontent.com/--JYWBYgNmA4/AAAAAAAAAAI/AAAAAAAAAAA/AMZuucla_xsuNyD0JzztPnDxxHGERfdcgg/photo.jpg",
  "userName": "Maria Rosa Ilardi"
}


`repliedAt` and `replyContent` contain the developer response to the review. Of course, they can be missing.

How many app reviews did we get?



In [ ]:
len(app_reviews)

20000

Let's save the reviews to a CSV file:

In [ ]:
if not os.path.isfile('balanced_reviews.csv'):
  balanced_reviews_df = pd.DataFrame(app_reviews)
  balanced_reviews_df.to_csv('balanced_reviews.csv', index=None, header=True, sep='\t')
  print('generato file balanced_reviews.csv')

balanced_reviews_df = pd.read_csv('balanced_reviews.csv', sep='\t')

generato file balanced_reviews.csv


In [ ]:
balanced_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              20000 non-null  object
 1   userName              20000 non-null  object
 2   userImage             20000 non-null  object
 3   content               20000 non-null  object
 4   score                 20000 non-null  int64 
 5   thumbsUpCount         20000 non-null  int64 
 6   reviewCreatedVersion  18913 non-null  object
 7   at                    20000 non-null  object
 8   replyContent          10116 non-null  object
 9   repliedAt             10116 non-null  object
 10  appId                 20000 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.7+ MB
